In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [11]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [12]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [13]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110320  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,7.519219e+05
2018-02-28,6.132667e+05
2018-03-31,8.486385e+05
2018-04-30,9.517111e+05
2018-05-31,9.671206e+05
2018-06-30,8.751299e+05
2018-07-31,9.861086e+05
2018-08-31,1.069968e+06
2018-09-30,9.808689e+05


In [14]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    7.519219e+05
 2018-02-28    6.132667e+05
 2018-03-31    8.486385e+05
 2018-04-30    9.517111e+05
 2018-05-31    9.671206e+05
 2018-06-30    8.751299e+05
 2018-07-31    9.861086e+05
 2018-08-31    1.069968e+06
 2018-09-30    9.808689e+05
 2018-10-31    9.225676e+05
 2018-11-30    9.489959e+05
 2018-12-31    9.581964e+05
 2019-01-31    1.062035e+06
 2019-02-28    8.496775e+05
 2019-03-31    1.097103e+06
 2019-04-30    9.478509e+05
 2019-05-31    1.041657e+06
 2019-06-30    9.557695e+05
 2019-07-31    9.778204e+05
 2019-08-31    1.112229e+06
 2019-09-30    8.295591e+05
 2019-10-31    9.343010e+05
 2019-11-30    9.522853e+05
 2019-12-31    1.088866e+06
 2020-01-31    9.084643e+05
 2020-02-29    7.741832e+05
 2020-03-31    8.300537e+05
 2020-04-30    8.944559e+05
 2020-05-31    8.347944e+05
 2020-06-30    8.553504e+05
 2020-07-31    7.349162e+05
 2020-08-31    9.792120e+05
 2020-09-30    7.782283e+05
 2020-10-31    8.875069e+05
 2020-11-30    9.742421e+05

# ARIMA

In [15]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [16]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -0.787850
p-value : 0.822665
Critical Values :
	1%: -3.620918
	5%: -2.943539
	10%: -2.610400


In [17]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -5.740344
p-value : 0.000001
Critical Values :
	1%: -3.620918
	5%: -2.943539
	10%: -2.610400


In [18]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1099.901, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1099.788, Time=0.03 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1098.003, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1096.084, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1091.512, Time=0.18 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1088.070, Time=0.20 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1097.626, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1099.397, Time=0.06 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1099.755, Time=0.07 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1101.532, Time=0.29 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1090.191, Time=0.28 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1097.591, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1098.959, Time=0.08 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1100.673, Time=0.17 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1099.815, Time=0.40 sec
 ARIMA(3,0

ARIMA(order=(0, 0, 5), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [19]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(0,0,5))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 5)   Log Likelihood                -537.263
Method:                       css-mle   S.D. of innovations          77555.776
Date:                Sat, 11 Sep 2021   AIC                           1088.527
Time:                        01:47:22   BIC                           1100.690
Sample:                             0   HQIC                          1092.985
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.992e+05   3.74e+04     24.066      0.000    8.26e+05    9.72e+05
ma.L1.y        0.2365      0.165      1.436      0.151      -0.086       0.559
ma.L2.y        0.5558      0.105      5.273      0.0

# 예측하기

In [20]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([903454.21486861, 899199.76996371]), array([77555.77552874, 79694.64970507]), array([[ 751447.68803922, 1055460.74169801],
       [ 743001.12678125, 1055398.41314618]]))
